In [23]:
import requests


client_id = "8eAtLJEwYIGbiA"


auth_url = "https://www.reddit.com/api/v1/authorize?client_id=8eAtLJEwYIGbiA&response_type=code&state=654321&redirect_uri=https://github.com/tjblue/WSBmonitor&duration=permanent&scope="

# TO DO
* Add rate limiting wait
* See if we can grab every comment from daily threads; pretty sure you can, not seeing more than a few comments every second, but have to double check
  * Only get new comments, compare with last check and only take the difference
* Comment filter for stock ticker mentions (possible setiment analysis on comment)
* In the process make a better live thread viewer, more linke twitch chat


In [477]:
from bs4 import BeautifulSoup
import re
import nltk

class WSBMonitor():
    
    def __init__(self):
        self.headers = {'user-agent': 'python:WSBmonitor:v0.1 (by /u/tjbluedev)'}
        self.sticky_rss_link = ""
        

    def getSticky(self):
        """
        Finds the link for the top stickied post for the discussion daily thread or "What are your moves" threads
        
        return: str
                    rss link to reddit post
        """
        
        main_rss_text = requests.get("https://www.reddit.com/r/wallstreetbets/.rss", headers=self.headers)

        regex_sticky = re.compile(r"(?<=/u/OPINION_IS_UNPOPULAR &lt;/a&gt; &lt;br/&gt; &lt;span&gt;&lt;a href=&quot;).*?(?=&quot;&gt)")
        sticky_rss_link = regex_sticky.findall(main_rss_text.text)[0] + ".rss"
        
        self.sticky_rss_link = sticky_rss_link
        return sticky_rss_link
    

    def getComments(self, url=""):
        """
        Finds the latest comments from a reddit url
        
        params: url: str
                    reddit url to get the latest comments from
        
        return: list 
                    list of newest comments posted at time of retrival
        """
        #Checking if url param is passed, if not url is defined by the sticky thread url found in findSticky().
        if url == "":
            url = self.sticky_rss_link
        
        #Requesting rss feed for url
        sticky_RSS_text = requests.get(url, headers=self.headers)
        soup = BeautifulSoup(sticky_RSS_text.text, "lxml")
        comments = soup.find_all("entry")
        
        #Stripping for content only
        stripped_list = []
        for i in range(len(comments)):
            stripped_list.append(comments[i].find("content").text[34:-25])
            
        return stripped_list
    
    def stripFormat(self, text):
        """
        Strips out reddit styling
        
        params: text: str
                    text to be stripped of html tags used for reddit styling
        
        return: str
        """
        #Special replace for superscript
        text = text.replace("<sup>", " ")
        #Replacing everything between "<" and ">"
        text = re.sub("<.*?>", "", text).replace("  ", " ")
        return text
        
        #Loops and replaces each tag with an empty string
        
        
    
    def isTicker(self, text):
        """
        Returns true or false if text is a ticker symbol
        
        params: text: str
                    text that will be cross checked with list of NYSE and NASDAQ stock ticker symbols
                    
        return: boolean
                    True or False
        """
        #Searches NYSE tickers
        with open("./tickers/NYSE.txt") as file:
            for line in file:
                if text.upper() == line.strip():
                    return True
                else:
                    continue
                    
        #Searches NASDAQ tickers            
        with open("./tickers/NASDAQ.txt") as file:
            for line in file:
                line.strip()
                if text.upper() == line.strip():
                    return True
                else:
                    continue
        return False
    
    def setimentAnalysis(self, text):
        pass   

In [430]:
#testing reddit formatting https://www.reddit.com/r/test/comments/lvjnmh/test/.rss
wsb = WSBmonitor()
test_rss = wsb.getComments("https://www.reddit.com/r/test/comments/lvjnmh/test/.rss")


TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [436]:
comments = wsb.getComments(sticky_thread)

In [480]:
import numpy as np
import pandas as pd
import nltk

class WSBTrain():
    
    def __init__(self):
        self.dataset = "./data/sortedData.csv"  
    
    def evalData(self, text):
        review = input(text + ". Is this (P)ositive 1 or (Neg)ative 2 or (Neu)tral? 3 ")
        if review == 1:
            review = 1
        elif review == 2: 
            review = -1
        elif review == 3:
            review = 0
        return [text, review]
    
    def storeData(self, entry):
        entry = entry[0] + "," + str(entry[1]) + "\n"
        with open(self.dataset, 'a') as file:
            file.write(entry)
        return
        
    def cleanNegations(self, text):
        pass



In [483]:
import time
train = WSBTrain()
wsb = WSBMonitor()

wsb.getSticky()
old_comments = wsb.getComments()
time.sleep(2)

while True:
    new_comments = wsb.getComments()
    net_comments = [comment for comment in new_comments if comment not in old_comments]
    for net in net_comments:
        train.evalData(net)
        train.storeData
        
    fuse = input("Check for more? Y/N: ")
    if fuse == "y" or fuse == "Y":
        old_comments = new_comments
        continue
    else:
        break

Opinions on Portnoy’s $BUZZ ETF?. Is this (P)ositive 1 or (Neg)ative 2 or (Neu)tral? 3  3
Like if spending stimi check on GME.... Is this (P)ositive 1 or (Neg)ative 2 or (Neu)tral? 3  3
The only stonks rn that will bring $$ is GME and RKT. Idgaf what anyone else is tooting about otherwise. Is this (P)ositive 1 or (Neg)ative 2 or (Neu)tral? 3  1
Check for more? Y/N:  y
I dont know how to read but I assume that rkt,sos,pltr and other means GME 🤣🚀🙌🏽💎. Is this (P)ositive 1 or (Neg)ative 2 or (Neu)tral? 3  3
C&#39;mon <a href="/u/opinion_is_unpopular">u/opinion_is_unpopular</a> and <a href="/u/zjz">u/zjz</a>, just be level with us and tell us why you won&#39;t make a GME thread. We aren&#39;t even asking for it to be pinned or anything, just give us our usual daily thread where we can comment on the stock and not spam the general daily thread. </p> <p>It comes of sus as all hell when you guys just don&#39;t do it and don&#39;t say shit, when the majority of your community is asking for it..

In [ ]:
negations_ = {"isn't":"is not", "can't":"can not","couldn't":"could not", "hasn't":"has not",
                "hadn't":"had not","won't":"will not",
                "wouldn't":"would not","aren't":"are not",
                "haven't":"have not", "doesn't":"does not","didn't":"did not",
                 "don't":"do not","shouldn't":"should not","wasn't":"was not", "weren't":"were not",
                "mightn't":"might not",
                "mustn't":"must not"}

In [450]:
#NLTK testing
from nltk.sentiment import SentimentIntensityAnalyzer
tokens = nltk.word_tokenize(comments[2])
tagged = nltk.pos_tag(tokens)
sia = SentimentIntensityAnalyzer()

In [454]:
sia.polarity_scores('This melt up from RKT and SOS has been just fucking peachy. Even made some easy money off 16000 shares of A A B B. </p> <p>Probably just hold and watch things move unless everything shits the bed. Then I&#39;ll sell on the way down like a big old paperhanded (but profit taking) bitch.')

{'neg': 0.112, 'neu': 0.717, 'pos': 0.171, 'compound': 0.3818}

In [445]:
tagged

[('I', 'PRP'),
 ('wish', 'VBP'),
 ('people', 'NNS'),
 ('on', 'IN'),
 ('this', 'DT'),
 ('sub', 'NN'),
 ('were', 'VBD'),
 ('as', 'IN'),
 ('excited', 'VBN'),
 ('about', 'RB'),
 ('FUBO', 'NNP'),
 ('as', 'IN'),
 ('they', 'PRP'),
 ('are', 'VBP'),
 ('about', 'IN'),
 ('GME', 'NNP'),
 (',', ','),
 ('RKT', 'NNP'),
 ('etc', 'FW'),
 ('.', '.'),
 ('😒', 'NN')]

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Blue\AppData\Roaming\nltk_data...


True